<a href="https://colab.research.google.com/github/awwnchal/Recommender-System/blob/main/Code_cf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# KNN Item-Based Collaborative Filtering

In [ ]:
hw4=pd.read_excel('/content/drive/MyDrive/Colab Notebooks/_Data_2022.xlsx', 'DBMI Data')
hw4=hw4.set_index('User')
hw4.head()

,The Social Network,A Prophet,Amour,The King's Speech,La La Land,Boyhood,Inception,A Separation,The Artist,The White Ribbon,...,Call Me by Your Name,Winter's Bone,The Grand Budapest Hotel,Dunkirk,Inside Llewyn Davis,A Serious Man,Toy Story 3,Beasts of the Southern Wild,The Imitation Game,The Fighter
User,,,,,,,,,,,,,,,,,,,,,
User 1,5.0,NaN,NaN,3.0,4.0,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User 2,3.0,NaN,NaN,4.0,5.0,4.0,5.0,NaN,NaN,NaN,...,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,5.0,NaN
User 3,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User 4,NaN,NaN,NaN,NaN,5.0,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
User 5,4.0,NaN,NaN,4.0,5.0,NaN,5.0,NaN,NaN,NaN,...,4.0,NaN,5.0,3.0,NaN,NaN,4.0,NaN,NaN,NaN


In [ ]:
hw4_1=pd.read_excel('/content/drive/MyDrive/Colab Notebooks/_Data_2022.xlsx', 'Form Responses 1')
hw4_1.head()

,Serial Number,Please write your first name below,Please write your last name below,"Please write your section number below (1,2)",Please write your group number or letter below,[The Social Network],[A Prophet],[Amour],[The King's Speech],[La La Land],...,[Call Me by Your Name],[Winter's Bone],[The Grand Budapest Hotel],[Dunkirk],[Inside Llewyn Davis],[A Serious Man],[Toy Story 3],[Beasts of the Southern Wild],[The Imitation Game],[The Fighter]
0,A,Shachi,Govil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B,Amy,Russell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C,Camille,Mack,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Teagan,Towhey,1.0,17,NaN,Did Not Watch,Did Not Watch,Did Not Watch,Did Not Watch,...,Did Not Watch,NaN,Did Not Watch,Did Not Watch,Did Not Watch,NaN,Did Not Watch,Did Not Watch,Did Not Watch,Did Not Watch
4,2,Anushka,Shah,1.0,18,4,Did Not Watch,Did Not Watch,Did Not Watch,Did Not Watch,...,Did Not Watch,NaN,Did Not Watch,5,Did Not Watch,NaN,3,Did Not Watch,Did Not Watch,Did Not Watch


# Data Cleaning

In [ ]:
drop_cols=hw4_1.columns[0:5].to_list()

In [ ]:
hw4_test=hw4_1.drop(drop_cols, axis=1)
hw4_test=hw4_test.replace('Did Not Watch', 0)
hw4_test=hw4_test.fillna(0)
hw4_test.head()

,[The Social Network],[A Prophet],[Amour],[The King's Speech],[La La Land],[Boyhood],[Inception],[A Separation],[The Artist],[The White Ribbon],...,[Call Me by Your Name],[Winter's Bone],[The Grand Budapest Hotel],[Dunkirk],[Inside Llewyn Davis],[A Serious Man],[Toy Story 3],[Beasts of the Southern Wild],[The Imitation Game],[The Fighter]
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0


In [ ]:
hw4=hw4.fillna(0)
hw4_t=hw4.T
hw4_t=hw4_t.fillna(0)

In [ ]:
hw4_t.head()
hw4_t.shape

(50, 280)

# Initial Training/Exploration

In [ ]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(hw4_t.values)
distances, indices = knn.kneighbors(hw4_t.values, n_neighbors=3)

In [ ]:
def generate_prediction(movie, user, n_neighbors, hw4_t):
  knn = NearestNeighbors(metric='cosine', algorithm='brute')
  knn.fit(hw4_t.values)

  # find the indices for the similar movies
  distances, indices = knn.kneighbors(hw4_t.values, n_neighbors=n_neighbors) 
  index_for_movie = hw4_t.index.tolist().index(movie)


 # find the indices for the similar movies
  sim_movies = indices[index_for_movie].tolist() #gives the indices of the similar movies along with the index of the movie passed in the function
  
  # distances between 'movie_0' and the similar movies
  movie_distances = distances[index_for_movie].tolist()

  # the position of 'movie_0' in the list sim_movies
  id_movie = sim_movies.index(index_for_movie)

  # remove the movie passed in the function from the list sim_movies
  sim_movies.remove(index_for_movie)

  # remove 'movie_0' from the list movie_distances
  movie_distances.pop(id_movie)
 
  nominator=0
  movie_similarity = [1-x for x in movie_distances]
  
  user_index = hw4_t.columns.tolist().index(user)
  #print(user_index)

  # for each similar movie
  for s in range(0, len(movie_similarity)):
      nominator = nominator + movie_similarity[s]*hw4_t.iloc[sim_movies[s],user_index]

  # check if the number of the ratings with non-zero is positive
  if len(movie_similarity) > 0:
    
    # check if the sum of the ratings of the similar movies is positive.
    if sum(movie_similarity) > 0:
      predicted_r = nominator/sum(movie_similarity)

    # Even if there are some movies for which the ratings are positive, some movies have zero similarity even though they are selected as similar movies.
    # in this case, the predicted rating becomes zero as well  
    else:
      predicted_r = 0

  # if all the ratings of the similar movies are zero, then predicted rating should be zero
  else:
    predicted_r = 0

  # place the predicted rating into the copy of the original dataset
  return predicted_r

In [ ]:
#hw4_t.iloc[3,7] #checking what iloc does #lala land and user 8 (cause starts from 0), user 8's rating for lalaland
hw4_t.head()

User,User 1,User 2,User 3,User 4,User 5,User 6,User 7,User 8,User 9,User 10,...,User 271,User 272,User 273,User 274,User 275,User 276,User 277,User 278,User 279,User 280
The Social Network,5.0,3.0,0.0,0.0,4.0,4.0,0.0,0.0,5.0,0.0,...,0.0,4.0,4.0,2.0,4.0,4.0,4.0,3.0,4.0,4.0
A Prophet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
Amour,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The King's Speech,3.0,4.0,0.0,0.0,4.0,0.0,5.0,0.0,5.0,0.0,...,3.0,0.0,5.0,0.0,0.0,3.0,0.0,4.0,0.0,5.0
La La Land,4.0,5.0,3.0,5.0,5.0,0.0,0.0,5.0,4.0,5.0,...,5.0,3.0,3.0,0.0,3.0,5.0,4.0,0.0,0.0,4.0


In [ ]:
df_train=hw4_t.copy() #m and i to keep count

# for each movie and user in the dataset
for m, t in list(enumerate(hw4_t.index)): 
  for i, k in list(enumerate(hw4_t.columns)):

    # if the rating is zero, then generate the predicted rating
    df_train.iloc[m, i]=generate_prediction(t, k, 15, hw4_t)

In [ ]:
import math  
import sklearn

# calculate the RMSE
mse = sklearn.metrics.mean_squared_error(hw4_t.values, df_train.values) 
rmse = math.sqrt(mse)
rmse

2.1873609910017753

In [ ]:
df_train.head()

User,User 1,User 2,User 3,User 4,User 5,User 6,User 7,User 8,User 9,User 10,...,User 271,User 272,User 273,User 274,User 275,User 276,User 277,User 278,User 279,User 280
The Social Network,2.423367,3.394028,0.207879,2.757527,2.474954,2.843447,1.140970,1.894539,4.060908,2.621525,...,3.051070,2.497223,4.153336,2.800455,3.388949,1.452495,3.512505,3.657308,1.081167,3.526869
A Prophet,1.224115,1.019698,0.252090,0.330579,0.264463,1.247577,0.660071,0.330579,1.090261,0.470996,...,1.897414,0.192309,1.892974,1.180201,0.651094,1.146754,0.811904,1.707479,0.000000,1.783174
Amour,1.377268,1.159232,0.178652,0.592424,1.115228,0.447307,0.732343,0.917739,1.082108,0.756689,...,1.725521,0.178652,1.484314,0.294671,0.473323,1.282611,0.738851,1.724024,0.000000,1.425101
The King's Speech,2.964500,3.358010,0.239925,2.823787,2.311870,2.846095,0.763339,1.966521,4.234309,2.415841,...,2.592578,2.381098,4.133271,2.619324,3.242397,1.954314,3.504299,3.478940,1.394318,3.920073
La La Land,2.489539,3.000696,0.000000,2.462418,2.723200,2.681503,1.159737,1.885316,3.779485,2.593874,...,2.496351,2.153144,4.025368,2.445007,3.034830,1.719129,3.480301,3.702386,1.417401,3.323927


In [ ]:
hw4.head()

,The Social Network,A Prophet,Amour,The King's Speech,La La Land,Boyhood,Inception,A Separation,The Artist,The White Ribbon,...,Call Me by Your Name,Winter's Bone,The Grand Budapest Hotel,Dunkirk,Inside Llewyn Davis,A Serious Man,Toy Story 3,Beasts of the Southern Wild,The Imitation Game,The Fighter
User,,,,,,,,,,,,,,,,,,,,,
User 1,5.0,0.0,0.0,3.0,4.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
User 2,3.0,0.0,0.0,4.0,5.0,4.0,5.0,0.0,0.0,0.0,...,0.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0
User 3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
User 4,0.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
User 5,4.0,0.0,0.0,4.0,5.0,0.0,5.0,0.0,0.0,0.0,...,4.0,0.0,5.0,3.0,0.0,0.0,4.0,0.0,0.0,0.0


In [ ]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(hw4.values)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
def generate_prediction_user(movie, user, n_neighbors, hw4):

  # fit the model
  knn = NearestNeighbors(metric='cosine', algorithm='brute')
  knn.fit(hw4.values)

  # find the indices for the similar movies
  distances, indices = knn.kneighbors(hw4.values, n_neighbors=n_neighbors)
  index_for_user = hw4.index.tolist().index(user)

  # distances between 'movie_0' and the similar movies
  user_distances = distances[index_for_user].tolist()

  # the indices for the similar movies
  sim_movies = indices[index_for_user].tolist()
  
  # the position of 'movie_0' in the list sim_movies
  id_movie = sim_movies.index(index_for_user)

  # remove 'movie_0' from the list sim_movies
  sim_movies.remove(index_for_user)
  
  # remove 'movie_0' from the list movie_distances
  user_distances.pop(id_movie)
  nominator=0
  user_similarity=[1-x for x in user_distances]

  movie_index = hw4.columns.tolist().index(movie)

  for s in range(0, len(user_similarity)):
    nominator = nominator + user_similarity[s]*hw4.iloc[sim_movies[s],movie_index]

  if len(user_similarity) > 0:
      
      # check if the sum of the ratings of the similar movies is positive.
      if sum(user_similarity) > 0:
        predicted_r = nominator/sum(user_similarity)

      # Even if there are some movies for which the ratings are positive, some movies have zero similarity even though they are selected as similar movies.
      # in this case, the predicted rating becomes zero as well  
      else:
        predicted_r = 0

    # if all the ratings of the similar movies are zero, then predicted rating should be zero
  else:
    predicted_r = 0

  return predicted_r

In [ ]:
# for each movie and user in the dataset
df_train_user=hw4.copy()
for m, t in list(enumerate(hw4.index)):
  for i,k in list(enumerate(hw4.columns)):
    df_train_user.iloc[m,i]=generate_prediction_user(k, t, 15, hw4)

In [ ]:
df_train_user.head()

,The Social Network,A Prophet,Amour,The King's Speech,La La Land,Boyhood,Inception,A Separation,The Artist,The White Ribbon,...,Call Me by Your Name,Winter's Bone,The Grand Budapest Hotel,Dunkirk,Inside Llewyn Davis,A Serious Man,Toy Story 3,Beasts of the Southern Wild,The Imitation Game,The Fighter
User,,,,,,,,,,,,,,,,,,,,,
User 1,3.001668,0.0,0.0,4.050361,3.462994,0.137540,4.635844,0.276403,0.000000,0.0,...,0.000000,0.000000,1.065336,0.787646,0.000000,0.212715,1.963343,0.0,2.448555,0.000000
User 2,3.929813,0.0,0.0,3.860339,4.000972,1.707712,4.859155,0.283849,0.639956,0.0,...,0.927045,0.000000,3.214921,3.721976,0.922552,0.283390,1.852187,0.0,4.002064,0.852090
User 3,2.697296,0.0,0.0,2.278285,2.856748,0.785787,4.273506,0.325434,1.200282,0.0,...,1.049349,0.361915,1.731468,2.978810,0.260347,0.000000,2.862478,0.0,4.193717,1.009515
User 4,1.536733,0.0,0.0,1.613227,4.063586,0.000000,3.967301,0.000000,0.208102,0.0,...,1.808952,0.000000,3.301964,3.223307,0.000000,0.000000,1.695078,0.0,2.310414,0.000000
User 5,2.675287,0.0,0.0,3.384566,4.500761,0.306807,3.936931,0.689617,0.000000,0.0,...,2.722408,0.153403,3.446528,1.577572,0.000000,0.000000,2.509403,0.0,2.240892,0.000000


In [ ]:
import math  
import sklearn
mse = sklearn.metrics.mean_squared_error(hw4.values, df_train_user.values) 
rmse = math.sqrt(mse)
rmse

1.1902882089289595

# Matrix Factorization

In [ ]:
data_mat=hw4.values
user_ratings_mean = np.mean(data_mat, axis =1)
mat_demeaned = data_mat- user_ratings_mean.reshape(-1, 1)

In [ ]:
from scipy.sparse.linalg import svds

# get the U, sigma, Vt
U, sigma, Vt = svds(mat_demeaned, k = 45)
sigma = np.diag(sigma)

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = hw4.columns)

In [ ]:
import math  
import sklearn
mse = sklearn.metrics.mean_squared_error(hw4.values, preds_df.values) 
rmse = math.sqrt(mse)
rmse

0.0949951567104564

# Testing the models

In [ ]:
hw4_test.columns = [
    c.replace(']', '').replace('[', '')
    for c in hw4.columns
]
hw4_test
#hw4_test.columns=[hw4_test.columns.str.replace(']','')]
#hw4_test
hw4_test=hw4_test.set_index(hw4_1['Please write your first name below']+' '+ hw4_1['Please write your last name below'])

In [ ]:
hw4_test=pd.concat([hw4_test, hw4])
hw4_test_t=hw4_test.T
hw4_test_t.head()

,0,1,2,3,4,5,6,7,8,9,...,User 271,User 272,User 273,User 274,User 275,User 276,User 277,User 278,User 279,User 280
[The Social Network],0.0,0.0,0.0,0.0,4.0,0.0,5.0,4.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[A Prophet],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[Amour],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[The King's Speech],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[La La Land],0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute')

,Shachi Govil,Amy Russell,Camille Mack,Teagan Towhey,Anushka Shah,Sanka Naga Nitesh Nitesh,Akash Puthalath,Arjun Mahesh,Parina Kolhe,Vindhya Mandekar,...,User 271,User 272,User 273,User 274,User 275,User 276,User 277,User 278,User 279,User 280
The Social Network,0.0,0.0,0.0,0.0,4.0,0.0,5.0,4.0,5.0,5.0,...,0.0,4.0,4.0,2.0,4.0,4.0,4.0,3.0,4.0,4.0
A Prophet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
Amour,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The King's Speech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,...,3.0,0.0,5.0,0.0,0.0,3.0,0.0,4.0,0.0,5.0
La La Land,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,5.0,...,5.0,3.0,3.0,0.0,3.0,5.0,4.0,0.0,0.0,4.0


## Non-Normalized

In [ ]:
def generate_preds_item(movie, user, n_neighbors):
  
  # fit the model
  knn = NearestNeighbors(metric='cosine', algorithm='brute')
  knn.fit(hw4_test_t.values)

  # find the indices for the similar movies
  distances, indices = knn.kneighbors(hw4_test_t.values, n_neighbors=n_neighbors)
  index_for_movie = hw4_test_t.index.tolist().index(movie)

  # find the indices for the similar movies
  sim_movies = indices[index_for_movie].tolist()

  # distances between 'movie_0' and the similar movies
  movie_distances = distances[index_for_movie].tolist()

  # the position of 'movie_0' in the list sim_movies
  id_movie = sim_movies.index(index_for_movie)

  # remove 'movie_0' from the list sim_movies
  sim_movies.remove(index_for_movie)
  
  # remove 'movie_0' from the list movie_distances
  movie_distances.pop(id_movie)
  nominator=0
  movie_similarity=[1-x for x in movie_distances]

  user_index = hw4_test_t.columns.tolist().index(user)

  # for each similar movie
  for s in range(0, len(movie_similarity)):
      nominator = nominator + movie_similarity[s]*hw4_test_t.iloc[sim_movies[s],user_index]

  # check if the number of the ratings with non-zero is positive
  if len(movie_similarity) > 0:
    
    # check if the sum of the ratings of the similar movies is positive.
    if sum(movie_similarity) > 0:
      predicted_r = nominator/sum(movie_similarity)

    # Even if there are some movies for which the ratings are positive, some movies have zero similarity even though they are selected as similar movies.
    # in this case, the predicted rating becomes zero as well  
    else:
      predicted_r = 0

  # if all the ratings of the similar movies are zero, then predicted rating should be zero
  else:
    predicted_r = 0

  # place the predicted rating into the copy of the original dataset
  return predicted_r

In [ ]:
print(generate_preds_item(' Son of Saul', 'Jake Brophy', 25))
generate_preds_item(" Son of Saul", "Jake Brophy", 25)

1.7436532292674198


1.7436532292674198

## Normalized

In [ ]:
data_mat=hw4_test_t.replace(0, np.nan)
m = data_mat.mean(axis=1) #column mean 
for i, col in enumerate(hw4_test_t.iloc[:,:2]):
    data_mat.iloc[:, i] = data_mat.iloc[:, i].fillna(m)
data_mat
user_ratings_mean = np.mean(data_mat, axis =1)
data_mat=data_mat.subtract(data_mat.mean(axis=1), axis = 0)
data_mat=data_mat.fillna(0)
data_mat.head()
#mat_demeaned = hw4_test_t- user_ratings_mean.reshape(-1, 1)

,0,1,2,3,4,5,6,7,8,9,...,User 271,User 272,User 273,User 274,User 275,User 276,User 277,User 278,User 279,User 280
[The Social Network],0.000000e+00,0.000000e+00,0.0,0.0,-0.122449,0.0,0.877551,-0.122449,0.877551,0.877551,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
[A Prophet],0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
[Amour],0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
[The King's Speech],0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0.000000,1.128205,1.128205,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
[La La Land],-4.440892e-16,-4.440892e-16,0.0,0.0,0.000000,0.0,-1.850746,0.000000,-1.850746,1.149254,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def generate_preds_item_mean(movie, user, n_neighbors, hw4_test_t):
  
  # fit the model
  knn = NearestNeighbors(metric='cosine', algorithm='brute')
  knn.fit(hw4_test_t.values)

  # find the indices for the similar movies
  distances, indices = knn.kneighbors(hw4_test_t.values, n_neighbors=n_neighbors)
  index_for_movie = hw4_test_t.index.tolist().index(movie)

  # find the indices for the similar movies
  sim_movies = indices[index_for_movie].tolist()

  # distances between 'movie_0' and the similar movies
  movie_distances = distances[index_for_movie].tolist()

  # the position of 'movie_0' in the list sim_movies
  id_movie = sim_movies.index(index_for_movie)

  # remove 'movie_0' from the list sim_movies
  sim_movies.remove(index_for_movie)
  
  # remove 'movie_0' from the list movie_distances
  movie_distances.pop(id_movie)
  nominator=0
  movie_similarity=[1-x for x in movie_distances]

  user_index = hw4_test_t.columns.tolist().index(user)

  # for each similar movie
  for s in range(0, len(movie_similarity)):
      nominator = nominator + movie_similarity[s]*hw4_test_t.iloc[sim_movies[s],user_index]

  # check if the number of the ratings with non-zero is positive
  if len(movie_similarity) > 0:
    
    # check if the sum of the ratings of the similar movies is positive.
    if sum(movie_similarity) > 0:
      predicted_r = user_ratings_mean[index_for_movie]+nominator/sum(movie_similarity) 
      #

    # Even if there are some movies for which the ratings are positive, some movies have zero similarity even though they are selected as similar movies.
    # in this case, the predicted rating becomes zero as well  
    else:
      predicted_r = 0

  # if all the ratings of the similar movies are zero, then predicted rating should be zero
  else:
    predicted_r = 0

  # place the predicted rating into the copy of the original dataset
  return predicted_r

Investigating New User Performance

In [ ]:
print(' Inception', generate_preds_item_mean(' Inception', "Shachi Govil", 30, data_mat))
print(' Avatar', generate_preds_item_mean(' Avatar', "Shachi Govil", 30, data_mat))
generate_preds_item_mean(' Inception', "Amy Russell", 30, data_mat)

 Inception 4.490384615384616
 Avatar 4.015151515151515


4.490384615384616

In [ ]:
data_mat.index

Index([' The Social Network', ' A Prophet', ' Amour', ' The King's Speech',
       ' La La Land', ' Boyhood', ' Inception', ' A Separation', ' The Artist',
       ' The White Ribbon', ' Zero Dark Thirty', ' Avatar', ' Spotlight',
       ' Precious', ' The Tree of Life', ' 12 Years a Slave',
       ' Blue is the Warmest Colour', ' Son of Saul', ' Up in the Air',
       ' Inglourious Basterds', ' Mad Max: Fury Road', ' Moonlight',
       ' Birdman', ' Manchester by the Sea', ' Lincoln', ' Hugo',
       ' Toni Erdmann', ' The Shape of Water',
       ' Three Billboards Outside Ebbing, Missouri', ' Argo', ' Gravity',
       ' Black Swan', ' Ida', ' Leviathan', ' The Wolf of Wall Street',
       ' True Grit', ' The Descendants', ' The Secret in Their Eyes',
       ' Life of Pi', ' Arrival', ' Call Me by Your Name', ' Winter's Bone',
       ' The Grand Budapest Hotel', ' Dunkirk', ' Inside Llewyn Davis',
       ' A Serious Man', ' Toy Story 3', ' Beasts of the Southern Wild',
       ' The Imi

In [ ]:
print(' Zero Dark Thirty', generate_preds_item_mean(' Zero Dark Thirty', "Jake Brophy", 30, data_mat))
print(generate_preds_item_mean(' Zero Dark Thirty', 'Anchal  Chaudhary ', 30, data_mat))
print(generate_preds_item_mean(' Zero Dark Thirty', 'Xu Zhang', 30, data_mat))
print(generate_preds_item_mean(' Zero Dark Thirty', 'Venkata Aravind Sampath Bhagavatula', 30, data_mat))

print(' Mad Max: Fury Road', generate_preds_item_mean(' Mad Max: Fury Road', "Jake Brophy", 30, data_mat))
print(generate_preds_item_mean(' Mad Max: Fury Road', 'Anchal  Chaudhary ', 30, data_mat))
print(generate_preds_item_mean(' Mad Max: Fury Road', 'Xu Zhang', 30, data_mat))
print(generate_preds_item_mean(' Mad Max: Fury Road', 'Venkata Aravind Sampath Bhagavatula', 30, data_mat))

print(' The Imitation Game', generate_preds_item_mean(' The Imitation Game', "Jake Brophy", 30, data_mat))
print(generate_preds_item_mean(' The Imitation Game', 'Jake Brophy', 30, data_mat))
print(generate_preds_item_mean(' The Imitation Game', 'Jake Brophy', 30, data_mat))
print(generate_preds_item_mean(' The Imitation Game', 'Jake Brophy', 30, data_mat))

print(' Winter\'s Bone',generate_preds_item_mean(' Winter\'s Bone', 'Jake Brophy', 30, data_mat))
print(generate_preds_item_mean(' Winter\'s Bone', 'Anchal  Chaudhary ', 30, data_mat))
print(generate_preds_item_mean(' Winter\'s Bone', 'Xu Zhang', 30, data_mat))
print(generate_preds_item_mean(' Winter\'s Bone', 'Venkata Aravind Sampath Bhagavatula', 30, data_mat))

print(' Son of Saul',generate_preds_item_mean(' Son of Saul', 'Jake Brophy', 30, data_mat))
print(generate_preds_item_mean(' Son of Saul', 'Anchal  Chaudhary ', 30, data_mat))
print(generate_preds_item_mean(' Son of Saul', 'Xu Zhang', 30, data_mat))
print(generate_preds_item_mean(' Son of Saul', 'Venkata Aravind Sampath Bhagavatula', 30, data_mat))

print(' A Serious Man',generate_preds_item_mean(' A Serious Man', 'Jake Brophy', 30, data_mat))
print(generate_preds_item_mean(' A Serious Man', 'Anchal  Chaudhary ', 30, data_mat))
print(generate_preds_item_mean(' A Serious Man', 'Xu Zhang', 30, data_mat))
print(generate_preds_item_mean(' A Serious Man', 'Venkata Aravind Sampath Bhagavatula', 30, data_mat))

 Zero Dark Thirty 4.126182367234396
3.918042441408359
3.4830744002312106
3.9486924239980103
 Mad Max: Fury Road 3.6521269985396354
3.633982098922873
3.087127993249225
3.6189560783386177
 The Imitation Game 4.364922416090241
4.364922416090241
4.364922416090241
4.364922416090241
 Winter's Bone 2.8710594316867946
2.7891935002753896
2.624704265523748
2.6879524210526466
 Son of Saul 3.3671205150074175
3.2264223266110332
3.1682507695763054
3.214642054526728
 A Serious Man 2.748522252077366
2.632829667494818
2.478139981091859
2.589226316770433


# Matrix factorization (item-item)

## Initial Model

In [ ]:
data_mat=hw4_test.replace(0, np.nan)
m = data_mat.mean(axis=0)
data_mat.loc[:'Camille Mack']=data_mat.apply(lambda x: x.head(3).fillna(x.mean()),axis=0)
R=data_mat.copy()
user_ratings_mean = np.nanmean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R_demeaned[np.isnan(R)] = 0
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 20)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = hw4_test.columns)
preds_df=preds_df.set_index(hw4_test.index)

names=["Jake Brophy", 'Anchal  Chaudhary ', 'Xu Zhang', 'Venkata Aravind Sampath Bhagavatula', 'Shachi Govil', 'Amy Russell', 'Camille Mack']
movies=[" Son of Saul", " Winter's Bone", " A Serious Man", ' The Imitation Game', " Dunkirk", " Zero Dark Thirty", " Avatar", " Inception", " The Wolf of Wall Street"]

results_df=pd.DataFrame(columns=movies, index=names)

for i in range(len(movies)):
  for t in range(len(names)):
    results_df.loc[names[t], movies[i]]=preds_df.loc[names[t],movies[i]]

results_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """


,Son of Saul,Winter's Bone,A Serious Man,The Imitation Game,Dunkirk,Zero Dark Thirty,Avatar,Inception,The Wolf of Wall Street
Jake Brophy,4.199794,4.220443,4.18585,4.80392,4.772057,4.672288,2.925742,5.08829,2.003515
Anchal Chaudhary,3.923632,3.890102,3.93982,4.21533,4.102533,3.705089,4.114527,4.805936,4.019258
Xu Zhang,1.749471,1.761916,1.758333,1.677799,2.0698,1.777908,0.976443,1.061368,1.904709
Venkata Aravind Sampath Bhagavatula,3.781365,3.802445,3.752791,3.897678,4.842317,4.161987,2.996363,3.971843,5.046111
Shachi Govil,3.779134,3.739275,3.762194,4.183538,3.949954,3.765666,3.935887,4.444184,4.059195
Amy Russell,3.779134,3.739275,3.762194,4.183538,3.949954,3.765666,3.935887,4.444184,4.059195
Camille Mack,3.779134,3.739275,3.762194,4.183538,3.949954,3.765666,3.935887,4.444184,4.059195


[link text](https://)## Without New information

In [ ]:
R=hw4_test.replace(0, np.nan).to_numpy()
user_ratings_mean = np.nanmean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R_demeaned[np.isnan(R)] = 0
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 20)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = hw4_test.columns)
preds_df=preds_df.set_index(hw4_test.index)

names=["Jake Brophy", 'Anchal  Chaudhary ', 'Xu Zhang', 'Venkata Aravind Sampath Bhagavatula', 'Shachi Govil', 'Amy Russell', 'Camille Mack']
movies=[" Son of Saul", " Winter's Bone", " A Serious Man", ' The Imitation Game', " Dunkirk", " Zero Dark Thirty", " Avatar", " Inception", " The Wolf of Wall Street"]

results_df=pd.DataFrame(columns=movies, index=names)

for i in range(len(movies)):
  for t in range(len(names)):
    results_df.loc[names[t], movies[i]]=preds_df.loc[names[t],movies[i]]

results_df

4.663247234977003
3.944176220228169
4.17709457460739
1.7552760712165927
3.7505703042877894
3.8935525932825303
4.213497581576377
1.7592926149449681
3.8001503641739123
3.9260106673467923
4.195767786931595
1.7478646836094183
3.779981998997083


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: Mean of empty slice
  


,Son of Saul,Winter's Bone,A Serious Man,The Imitation Game,Dunkirk,Zero Dark Thirty,Avatar,Inception,The Wolf of Wall Street
Jake Brophy,4.195768,4.213498,4.177095,4.803538,4.771874,4.663247,2.929224,5.089442,2.009423
Anchal Chaudhary,3.926011,3.893553,3.944176,4.221684,4.093868,3.703362,4.113756,4.799148,4.011513
Xu Zhang,1.747865,1.759293,1.755276,1.674751,2.071603,1.780871,0.974999,1.061565,1.905258
Venkata Aravind Sampath Bhagavatula,3.779982,3.80015,3.75057,3.892395,4.840539,4.160006,2.995471,3.971884,5.046988
Shachi Govil,4.192278,4.178563,4.188483,4.254823,4.202679,4.243459,4.232074,4.177216,4.209444
Amy Russell,3.199985,3.210926,3.23001,3.089497,3.210686,3.147869,3.171662,3.241959,3.2053
Camille Mack,2.803394,2.812233,2.795904,2.747588,2.839608,2.77834,2.729097,2.798115,2.757855


## Addition of New Information

In [ ]:
hw4_test.loc['Camille Mack',' Precious']=2.0
hw4_test.loc['Shachi Govil',' Precious']=4.0
hw4_test.loc['Amy Russell',' Precious']=4.0

hw4_test.loc['Camille Mack',' 12 Years a Slave']=2.0
hw4_test.loc['Shachi Govil',' 12 Years a Slave']=5.0
hw4_test.loc['Amy Russell',' 12 Years a Slave']=3.0

hw4_test.loc['Camille Mack',' Mad Max: Fury Road']=4.0
hw4_test.loc['Shachi Govil',' Mad Max: Fury Road']=5.0
hw4_test.loc['Amy Russell',' Mad Max: Fury Road']=4.0

hw4_test.loc['Camille Mack',' Black Swan']=3.0
hw4_test.loc['Shachi Govil',' Black Swan']=4.0
hw4_test.loc['Amy Russell',' Black Swan']=1.0

hw4_test.loc['Camille Mack',' Toy Story 3']=3.0
hw4_test.loc['Shachi Govil',' Toy Story 3']=3.0
hw4_test.loc['Amy Russell',' Toy Story 3']=4.0

In [ ]:
R=hw4_test.replace(0, np.nan).to_numpy()
user_ratings_mean = np.nanmean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R_demeaned[np.isnan(R)] = 0
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 20)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = hw4_test.columns)
preds_df=preds_df.set_index(hw4_test.index)

names=["Jake Brophy", 'Anchal  Chaudhary ', 'Xu Zhang', 'Venkata Aravind Sampath Bhagavatula', 'Shachi Govil', 'Amy Russell', 'Camille Mack']
movies=[" Son of Saul", " Winter's Bone", " A Serious Man", ' The Imitation Game', " Dunkirk", " Zero Dark Thirty", " Avatar", " Inception", " The Wolf of Wall Street"]

results_df=pd.DataFrame(columns=movies, index=names)

for i in range(len(movies)):
  for t in range(len(names)):
    results_df.loc[names[t], movies[i]]=preds_df.loc[names[t],movies[i]]

results_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: Mean of empty slice
  


,Son of Saul,Winter's Bone,A Serious Man,The Imitation Game,Dunkirk,Zero Dark Thirty,Avatar,Inception,The Wolf of Wall Street
Jake Brophy,4.195768,4.213498,4.177095,4.803538,4.771874,4.663247,2.929224,5.089442,2.009423
Anchal Chaudhary,3.926011,3.893553,3.944176,4.221684,4.093868,3.703362,4.113756,4.799148,4.011513
Xu Zhang,1.747865,1.759293,1.755276,1.674751,2.071603,1.780871,0.974999,1.061565,1.905258
Venkata Aravind Sampath Bhagavatula,3.779982,3.80015,3.75057,3.892395,4.840539,4.160006,2.995471,3.971884,5.046988
Shachi Govil,4.192278,4.178563,4.188483,4.254823,4.202679,4.243459,4.232074,4.177216,4.209444
Amy Russell,3.199985,3.210926,3.23001,3.089497,3.210686,3.147869,3.171662,3.241959,3.2053
Camille Mack,2.803394,2.812233,2.795904,2.747588,2.839608,2.77834,2.729097,2.798115,2.757855


In [ ]:
preds_df.columns.values

array([' The Social Network', ' A Prophet', ' Amour',
       " The King's Speech", ' La La Land', ' Boyhood', ' Inception',
       ' A Separation', ' The Artist', ' The White Ribbon',
       ' Zero Dark Thirty', ' Avatar', ' Spotlight', ' Precious',
       ' The Tree of Life', ' 12 Years a Slave',
       ' Blue is the Warmest Colour', ' Son of Saul', ' Up in the Air',
       ' Inglourious Basterds', ' Mad Max: Fury Road', ' Moonlight',
       ' Birdman', ' Manchester by the Sea', ' Lincoln', ' Hugo',
       ' Toni Erdmann', ' The Shape of Water',
       ' Three Billboards Outside Ebbing, Missouri', ' Argo', ' Gravity',
       ' Black Swan', ' Ida', ' Leviathan', ' The Wolf of Wall Street',
       ' True Grit', ' The Descendants', ' The Secret in Their Eyes',
       ' Life of Pi', ' Arrival', ' Call Me by Your Name',
       " Winter's Bone", ' The Grand Budapest Hotel', ' Dunkirk',
       ' Inside Llewyn Davis', ' A Serious Man', ' Toy Story 3',
       ' Beasts of the Southern Wild', ' 